An ipywidget interface to move incorrect predictions within the validation dataset should they be labeled wrong

In [1]:
import sys

sys.path.insert(0, "..")
import itertools


import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import torch

import cocpit
from cocpit import config as config

%load_ext autoreload
%autoreload 2

### check classifications from specific model and validation dataloader

In [15]:
#  Make predictions on each batch of images,
#  concatenate the precitions across batches,
#  and return the wrong predictions.
#  Loops over folds if using k-fold cross validation

# sourcery skip: use-itertools-product
p = cocpit.predictions.LoaderPredictions()
with torch.no_grad():
    for fold in range(config.KFOLD+1):
        for ((imgs, labels, paths), _) in p.load_val_loader(fold):
            b = cocpit.predictions.BatchPredictions(imgs, p.load_model(fold))
            b.find_max_preds()
            b.top_k_preds(len(config.CLASS_NAMES))
            p.append_batch(b, paths, labels)

p.concatenate_loader_vars()
p.find_wrong_indices()

print("DONE FINDING INCORRECT PREDICTIONS!")                
print(f"There are {len(p.labels)} total images in the dataloader.")
print(f"There are {len(p.wrong_idx)} wrong predictions to check in the dataloader!")


DONE FINDING INCORRECT PREDICTIONS!
There are 4308 total images in the dataloader.
There are 1767 wrong predictions to check in the dataloader!


In [16]:
# to only look between specific categories run this cell
label_list = dict(zip(cocpit.config.CLASS_NAMES, np.arange(0, len(cocpit.config.CLASS_NAMES))))

# change these two lines to focus on wrong predictions from a specific category
# or box within the confusion matrix
# makes labeling easier focusing on two at a time
human_label = label_list ["compact irregular"]
model_label = label_list["aggregate"]

p.hone_incorrect_predictions(label_list, human_label, model_label)

12 wrong predictions between compact irregular and aggregate


In [21]:
"""
Display a bar chart of model prediction probabilities
A dropdown menu is available to move the image to a different category in the training dataset
If you don't want to move the image and the human labeled correctly, simply click "Next"
"""
g = cocpit.gui_wrong.GUI(p.wrong_trunc, p.labels, p.paths, p.topk_probs, p.topk_classes)
g.make_buttons()
g.align_buttons()
g.visualizations()
layout=ipywidgets.Layout(display='flex', width='100%', align_items='center')
display(ipywidgets.VBox([ipywidgets.Box([g.center]), g.label_btns, g.next_btn], layout=layout))

In [22]:
# clear out the incorrect predictions between a specific scenario for human and model label.
# If this cell is run after all images have been iterated through, run the second 
# to last cell again to hone in on a different set of human and model predictions
p.wrong_trunc=[]